In [12]:
import sys
sys.path.append('../_amigocloud')

In [13]:
import collections
from amigocloud import AmigoCloud

from datetime import datetime

from docxtpl import DocxTemplate
import docxtpl
from docx.shared import Mm
import requests

import qrcode

In [14]:
# ruta de carpera donde estan las llaves
ruta = open('../ruta.txt')
ruta = ruta.readlines()[0]
# leer token
acceso = open(ruta + '/_keys/api_amigocloud.key','r')
api_token = acceso.readlines()[0]
acceso.close()
amigocloud = AmigoCloud(token=api_token)
amigocloud

In [15]:
# variables globales
id_proyecto = 34080

## convertir formato de fecha

In [16]:
# convierte de formato YYYY-mm-dd H:M:S+z a d/m/YYYY
def convertir_formato_fecha(fecha):
    new_formato = datetime.strptime(fecha, "%Y-%m-%d %H:%M:%S%z").strftime("%d/%m/%Y")
    return new_formato

## ejecutar query

In [17]:
# ejecuta cualquier query sql en el proyecto que se le indique
# requiere el id de proyecto, query a ejecutar y tipo solicitud (get o post)
def ejecutar_query_sql(id_project, query, tipo_sql):
    # define la url del proyecto para ejecutar el querry
    url_proyecto_sql = f'https://app.amigocloud.com/api/v1/projects/{id_project}/sql'
    # crea la estructura de query para amigocloud
    query_sql = {'query': query}
    
    resultado_get = ''
    # eleige que tipo de solicitud se realizara (get o post)
    if tipo_sql == 'get': 
        resultado_get = amigocloud.get(url_proyecto_sql, query_sql)
    elif tipo_sql == 'post':
        resultado_get = amigocloud.post(url_proyecto_sql, query_sql)
    else:
        resultado_get = 'Se a seleccionado un tipo de solicitud erroneo.'
    return resultado_get

In [18]:
# ejecuta un query que esta almacenado en un proyecto de amigocloud (generalmente un update),
# requiere id de proyecto e id de query
# retorna cuantas filas fueron afectadas
def ejecutar_query_por_id(id_project, id_query):
    # obtiene el query basado en el id_project y el id_query
    get_query = amigocloud.get(f'https://app.amigocloud.com/api/v1/projects/{id_project}/queries/{id_query}')
    # se extrae solo el texto del query
    query = get_query['query']
    # ejecuta el query_sql con metodo post y guarda la respuesta
    respuesta_post = ejecutar_query_sql(id_proyecto, query, 'post')
    # retorna el numero de filas afectadas por el query
    return respuesta_post['count']

## generar qr

In [19]:
# genera un qr con datos texto que le se envie
# requiere le texto de contenido, ruta donde se guardara el qr y nombre para guardar
def generar_qr(contenido_qr, ruta_save, id_name):
    # declara obj del qrocode
    qr = qrcode.QRCode(
        version=1,
        error_correction=qrcode.constants.ERROR_CORRECT_L,
        box_size=10,
        border=4)
    # se agrega el contenido del qr
    qr.add_data(contenido_qr)
    qr.make(fit=True)
    # define colores del qr
    img = qr.make_image(fill_color="black", back_color="white")
    # define ruta y nombre para guardar el qr
    ruta_qr = ruta_save + id_name + '.png'
    # guardar el código QR en un archivo .png
    img.save(ruta_qr)
    # retorna la ruta del qr
    return ruta_qr

In [20]:
id_insp = 20

In [21]:
# consulta pata extraer los datos de inspeccion por id concreto
query = f'select ca.canhero, ca.maquina, ca.modelo, ca.codigo_maquina, ca.frente, insp.* from dataset_341171 insp\
    inner join dataset_341170 ca on ca.amigo_id=insp.maquina_ref_preven\
    where insp.id={id_insp}'
select = ejecutar_query_sql(id_proyecto, query, 'get')
data = select['data'][0]

In [23]:
# modificacion de la fecha a formato dd/mm/aaaa
data['fecha_registro'] = convertir_formato_fecha(data['fecha_registro'])

In [27]:
# modificacion del nombre de las claves, con una k_ por delante, por que los campos booblenos (1_1, 2_2, etc) no se pueden trabajar bien
data = {'k_'+key : value for (key, value) in data.items()}
insp = collections.namedtuple("insp", data.keys())(*data.values())

In [84]:
id = 20

In [85]:
# consulta pata extraer los datos de inspeccion por id concreto
query = {'query': 'select ca.canhero, ca.maquina, ca.modelo, ca.codigo_maquina, ca.frente, insp.* from dataset_341171 insp\
    inner join dataset_341170 ca on ca.amigo_id=insp.maquina_ref_preven\
    where insp.id={id_insp}'.format(id_insp=id)}
project = 'https://app.amigocloud.com/api/v1/projects/34080/sql'
select = amigocloud.get(project, query)
data = select['data'][0]

In [299]:
# modificacion de la fecha a formato dd/mm/aaaa
data['fecha_registro'] = datetime.strptime(data['fecha_registro'], "%Y-%m-%d %H:%M:%S%z").strftime("%d/%m/%Y")
# modificacion del nombre de las claves, con una k_ por delante, por que los campos booblenos (1_1, 2_2, etc) no se pueden trabajar bien
data = {'k_'+key : value for (key, value) in data.items()}

insp = collections.namedtuple("insp", data.keys())(*data.values())

In [300]:
insp

insp(k_canhero='Victor Hugo escobar', k_maquina='Jhon Deere ', k_modelo='3520', k_codigo_maquina=None, k_frente=None, k_fecha_registro='04/04/2024', k_amigo_id='af80337a04b24a389269b89e86d4e960', k_id=20, k_usuario='Juan Pablo Jaldin Duran', k_obs=None, k_ubicacion='0101000020E61000004E85D3DDD09F4FC0F6C5EFB7875431C0', k_1_1=False, k_1_2=False, k_1_3=True, k_1_4=True, k_1_5=True, k_2_1=True, k_2_2=True, k_2_3=True, k_2_4=True, k_3_1=True, k_3_2=True, k_3_3=True, k_3_4=True, k_3_5=True, k_3_6=True, k_3_7=True, k_3_8=True, k_4_1=True, k_4_2=True, k_4_3=True, k_4_4=True, k_4_5=True, k_5_1=True, k_5_2=True, k_5_3=True, k_5_4=True, k_5_5=True, k_5_6=True, k_6_1=False, k_6_2=True, k_6_3=True, k_6_4=True, k_6_5=False, k_7_1=False, k_7_2=True, k_7_3=True, k_7_4=True, k_7_5=True, k_7_6=True, k_7_7=True, k_7_8=True, k_8_1=True, k_8_2=True, k_8_3=True, k_8_4=True, k_8_5=True, k_8_6=True, k_8_7=True, k_8_8=True, k_8_9=True, k_9_1=True, k_9_2=True, k_9_3=True, k_9_4=True, k_9_5=True, k_9_6=True, k_1

In [301]:
query_fotos = {'query': 'select * from gallery_57490 where source_amigo_id=\'{id}\''.format(id=insp.k_amigo_id)}
project = 'https://app.amigocloud.com/api/v1/projects/34080/sql'
fotos = amigocloud.get(project, query_fotos)['data']

In [291]:
#fotos

In [302]:
doc = DocxTemplate(ruta + '/templates/tpl_insp_cosechadoras-preventivo_2.docx')

In [303]:
lista_fotos_inline = []
for foto in fotos:
    response = requests.get("https://www-amigocloud.s3.amazonaws.com/gallery/" + foto['s3_filename'])
    file = open(ruta + '/fotos/' + foto['s3_filename'], "wb")
    file.write(response.content)
    file.close()
    lista_fotos_inline.append(docxtpl.InlineImage(doc, image_descriptor= ruta + '/fotos/' + foto['s3_filename'], width=Mm(50)))

In [304]:
len(lista_fotos_inline)

5

In [305]:
insp

insp(k_canhero='Victor Hugo escobar', k_maquina='Jhon Deere ', k_modelo='3520', k_codigo_maquina=None, k_frente=None, k_fecha_registro='04/04/2024', k_amigo_id='af80337a04b24a389269b89e86d4e960', k_id=20, k_usuario='Juan Pablo Jaldin Duran', k_obs=None, k_ubicacion='0101000020E61000004E85D3DDD09F4FC0F6C5EFB7875431C0', k_1_1=False, k_1_2=False, k_1_3=True, k_1_4=True, k_1_5=True, k_2_1=True, k_2_2=True, k_2_3=True, k_2_4=True, k_3_1=True, k_3_2=True, k_3_3=True, k_3_4=True, k_3_5=True, k_3_6=True, k_3_7=True, k_3_8=True, k_4_1=True, k_4_2=True, k_4_3=True, k_4_4=True, k_4_5=True, k_5_1=True, k_5_2=True, k_5_3=True, k_5_4=True, k_5_5=True, k_5_6=True, k_6_1=False, k_6_2=True, k_6_3=True, k_6_4=True, k_6_5=False, k_7_1=False, k_7_2=True, k_7_3=True, k_7_4=True, k_7_5=True, k_7_6=True, k_7_7=True, k_7_8=True, k_8_1=True, k_8_2=True, k_8_3=True, k_8_4=True, k_8_5=True, k_8_6=True, k_8_7=True, k_8_8=True, k_8_9=True, k_9_1=True, k_9_2=True, k_9_3=True, k_9_4=True, k_9_5=True, k_9_6=True, k_1

In [306]:
# Datos que quieres que contenga el QR
texto = '''
--UTEA--
--PROGRAMA CAÑA LIMPIA--

Id de Evaluacion: {nro_reg}
Fecha de Evaluacion: {fecha}
Nombre de Canero: {canero}
Codigo Canero: {cod_ca}
Maquina Cosechadora: {maquina}
Codigo de Maquina: {cod_ma}
Calificacion de Evaluacion: {nota}%'''\
.format(
    nro_reg=insp.k_id, 
    fecha=insp.k_fecha_registro, 
    canero=insp.k_canhero.split(' / ')[1],
    cod_ca=insp.k_canhero.split(' / ')[0],
    maquina=insp.k_maquina + '-' + insp.k_modelo,
    cod_ma=insp.k_codigo_maquina,
    nota=round(insp.k_nota_porcen*100, 2))
qr = qrcode.QRCode(
    version=1,
    error_correction=qrcode.constants.ERROR_CORRECT_L,
    box_size=10,
    border=4)
qr.add_data(texto)
qr.make(fit=True)

img = qr.make_image(fill_color="black", back_color="white")
#filename_qr = 'qr_' + str(insp.k_canhero.split(' / ')[0]) + '_' + str(data_rep['cod_prop']) + '_' + data_rep['fecha']
ruta_qr = ruta + '/planos/' + insp.k_amigo_id + '.png'
# Guardar el código QR en un archivo
img.save(ruta_qr)

qr_file = docxtpl.InlineImage(doc, image_descriptor=ruta_qr, width=Mm(30))

IndexError: list index out of range

In [281]:
context = {'insp':insp, 'fotos':lista_fotos_inline, 'qr':qr_file}

In [282]:
doc.render(context)

In [283]:
cod_nom = insp.k_canhero.split(' / ')
file_name = cod_nom[0] + '_ECP_' + insp.k_fecha_registro.replace('/','-') + '_' + cod_nom[1] + '_' + str(insp.k_id)

In [284]:
file_name

'3378_ECP_04-04-2024_ESCOBAR MORATO VICTOR HUGO_19'

In [285]:
doc.save(ruta + r'/informes/_' + file_name + '.docx')